In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [61]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from ipywidgets import interact,interact_manual
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

In [10]:
data=pd.read_csv("/content/drive/MyDrive/datasets/newdata1/Cleaned_WeatherAUS.csv",parse_dates=[1])
data.head()

,Unnamed: 0,Date,Location,MinTemp,MaxTemp,Rainfall,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,0,2008-12-01,Albury,13.4,22.9,0.6,8.4,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,4.0,16.9,21.8,No,No
1,1,2008-12-02,Albury,7.4,25.1,0.0,8.4,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,5.0,4.0,17.2,24.3,No,No
2,2,2008-12-03,Albury,12.9,25.7,0.0,8.4,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,5.0,2.0,21.0,23.2,No,No
3,3,2008-12-04,Albury,9.2,28.0,0.0,8.4,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,5.0,4.0,18.1,26.5,No,No
4,4,2008-12-05,Albury,17.5,32.3,1.0,8.4,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [11]:
data.drop(columns=["Unnamed: 0"],inplace=True)

In [12]:
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,8.4,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,4.0,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,8.4,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,5.0,4.0,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,8.4,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,5.0,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,8.4,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,5.0,4.0,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,8.4,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [14]:
data["year"]=data["Date"].dt.year

In [15]:
data=data.drop(columns=["Date"])

In [16]:
data.head()

,Location,MinTemp,MaxTemp,Rainfall,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,year
0,Albury,13.4,22.9,0.6,8.4,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,4.0,16.9,21.8,No,No,2008
1,Albury,7.4,25.1,0.0,8.4,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,5.0,4.0,17.2,24.3,No,No,2008
2,Albury,12.9,25.7,0.0,8.4,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,5.0,2.0,21.0,23.2,No,No,2008
3,Albury,9.2,28.0,0.0,8.4,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,5.0,4.0,18.1,26.5,No,No,2008
4,Albury,17.5,32.3,1.0,8.4,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No,2008


In [28]:
data[data.select_dtypes("object").columns]=data[data.select_dtypes("object").columns].apply(LabelEncoder().fit_transform)

In [29]:
data.head()

,Location,MinTemp,MaxTemp,Rainfall,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,year
0,1,13.4,22.9,0.6,8.4,13,44.0,13,14,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,4.0,16.9,21.8,0,0,2008
1,1,7.4,25.1,0.0,8.4,14,44.0,6,15,4.0,22.0,44.0,25.0,1010.6,1007.8,5.0,4.0,17.2,24.3,0,0,2008
2,1,12.9,25.7,0.0,8.4,15,46.0,13,15,19.0,26.0,38.0,30.0,1007.6,1008.7,5.0,2.0,21.0,23.2,0,0,2008
3,1,9.2,28.0,0.0,8.4,4,24.0,9,0,11.0,9.0,45.0,16.0,1017.6,1012.8,5.0,4.0,18.1,26.5,0,0,2008
4,1,17.5,32.3,1.0,8.4,13,41.0,1,7,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0,0,2008


In [33]:
data[["Pressure9am","Pressure3pm"]]=data[["Pressure9am","Pressure3pm"]].apply(lambda f: f.apply(lambda x: np.sqrt(abs(x))))

In [34]:
data.head()

,Location,MinTemp,MaxTemp,Rainfall,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,year
0,1,13.4,22.9,0.6,8.4,13,44.0,13,14,20.0,24.0,71.0,22.0,31.744291,31.734839,8.0,4.0,16.9,21.8,0,0,2008
1,1,7.4,25.1,0.0,8.4,14,44.0,6,15,4.0,22.0,44.0,25.0,31.789936,31.745866,5.0,4.0,17.2,24.3,0,0,2008
2,1,12.9,25.7,0.0,8.4,15,46.0,13,15,19.0,26.0,38.0,30.0,31.742716,31.760038,5.0,2.0,21.0,23.2,0,0,2008
3,1,9.2,28.0,0.0,8.4,4,24.0,9,0,11.0,9.0,45.0,16.0,31.899843,31.824519,5.0,4.0,18.1,26.5,0,0,2008
4,1,17.5,32.3,1.0,8.4,13,41.0,1,7,7.0,20.0,82.0,33.0,31.793081,31.717503,7.0,8.0,17.8,29.7,0,0,2008


In [36]:
data["RainTomorrow"].value_counts()

0    102819
1     28805
Name: RainTomorrow, dtype: int64

## From here we see that this is imbalanced Type of Machine learning if we go on 

In [65]:
x=data.drop(columns=["RainTomorrow"]).values
y=data["RainTomorrow"].values

In [66]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [67]:
@interact_manual
def Models(Model=[LogisticRegression(),RandomForestClassifier(),LinearDiscriminantAnalysis()]):
  model=Model
  model.fit(x_train,y_train)
  pred=model.predict(x_test)
  plt.rcParams["figure.figsize"]=(15,5)
  return (
      print(f'Accuracy with {Model} is {model.score(x_test,y_test)}'),
      print(" "),
      print(sklearn.metrics.classification_report(y_test,pred)),
      plt.subplot(1,2,1),
      sns.countplot(y_test,color="green"),
      plt.title("Actual Data"),
      plt.subplot(1,2,2),
      sns.countplot(pred,color="red"),
      plt.title("Predicted Data"),
      plt.show()
  )

interactive(children=(Dropdown(description='Model', options=(LogisticRegression(), RandomForestClassifier(), L…

# Now After Balance the data 

In [69]:
from imblearn.combine import SMOTEENN

In [70]:
re_x,re_y=SMOTEENN().fit_resample(x,y)

In [73]:
import pickle
pickle.dump(re_x,open("/content/drive/MyDrive/datasets/newdata1/RE_X.pkl","wb"))
pickle.dump(re_y,open("/content/drive/MyDrive/datasets/newdata1/RE_Y.pkl","wb"))

In [74]:
x_train,x_test,y_train,y_test=train_test_split(re_x,re_y,test_size=0.2)

# Logistic Regression Model

In [75]:
model=LogisticRegression()
model.fit(x_train,y_train)
print(f'Accuracy is {model.score(x_test,y_test)}')
print(" ")
print(classification_report(model.predict(x_test),y_test))

Accuracy is 0.8508007172447907
 
              precision    recall  f1-score   support

           0       0.83      0.81      0.82     13557
           1       0.87      0.88      0.87     18789

    accuracy                           0.85     32346
   macro avg       0.85      0.85      0.85     32346
weighted avg       0.85      0.85      0.85     32346



# LDA Model

In [76]:
model=LinearDiscriminantAnalysis()
model.fit(x_train,y_train)
print(f'Accuracy is {model.score(x_test,y_test)}')
print(" ")
print(classification_report(model.predict(x_test),y_test))

Accuracy is 0.860260928708341
 
              precision    recall  f1-score   support

           0       0.86      0.81      0.83     14009
           1       0.86      0.89      0.88     18337

    accuracy                           0.86     32346
   macro avg       0.86      0.85      0.86     32346
weighted avg       0.86      0.86      0.86     32346



# Random Forest Model

In [77]:
model=RandomForestClassifier()
model.fit(x_train,y_train)
print(f'Accuracy is {model.score(x_test,y_test)}')
print(" ")
print(classification_report(model.predict(x_test),y_test))

Accuracy is 0.9508130835342855
 
              precision    recall  f1-score   support

           0       0.94      0.94      0.94     13324
           1       0.96      0.96      0.96     19022

    accuracy                           0.95     32346
   macro avg       0.95      0.95      0.95     32346
weighted avg       0.95      0.95      0.95     32346



In [78]:
pickle.dump(model,open("/content/drive/MyDrive/datasets/newdata1/RFC_Model.pkl","wb"))